# Make Neut Curves and Analyze Mouse Neuts

In [ ]:
import math

import pandas as pd
import numpy as np
import seaborn as sns

import neutcurve

from plotnine import *

import csv
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.font_manager import FontProperties

import scipy.stats

from IPython.display import display, HTML

In [ ]:
CBP = neutcurve.colorschemes.CBPALETTE
CBM = neutcurve.colorschemes.CBMARKERS
theme_set(theme_seaborn(style='white', context='talk', font_scale=1))

In [ ]:
fidfs = {}
fidfs['rnd1'] = pd.read_csv('./fract_infect/201015_fractinfect.csv').drop(['Unnamed: 0'], axis=1)
fidfs['rnd2'] = pd.read_csv('./fract_infect/201016_fractinfect_rnd2.csv').drop(['Unnamed: 0'], axis=1)
fidfs['rnd3'] = pd.read_csv('./fract_infect/201017_fractinfect_rnd3.csv').drop(['Unnamed: 0'], axis=1)
fidfs['rnd4'] = pd.read_csv('./fract_infect/201020_fractinfect_rnd4.csv').drop(['Unnamed: 0'], axis=1)
fidfs['rnd5'] = pd.read_csv('./fract_infect/201021_fractinfect_rnd5.csv').drop(['Unnamed: 0'], axis=1)
fidfs['rnd6'] = pd.read_csv('./fract_infect/201022_fractinfect_rnd6.csv').drop(['Unnamed: 0'], axis=1)

In [ ]:
for fidf in fidfs.keys():
    display(HTML(fidfs[fidf].head().to_html(index=False)))

In [ ]:
smpls = {}
smpls['rnd1'] = pd.read_csv('./sample_maps/rnd1_sample_map.csv').rename(columns={'Sample': 'serum'})
smpls['rnd2'] = pd.read_csv('./sample_maps/rnd2_sample_map.csv').rename(columns={'Sample': 'serum'})
smpls['rnd3'] = pd.read_csv('./sample_maps/rnd3_sample_map.csv').rename(columns={'Sample': 'serum'})
smpls['rnd4'] = pd.read_csv('./sample_maps/rnd4_sample_map.csv').rename(columns={'Sample': 'serum'})
smpls['rnd5'] = pd.read_csv('./sample_maps/rnd5_sample_map.csv').rename(columns={'Sample': 'serum'})
smpls['rnd6'] = pd.read_csv('./sample_maps/rnd6_sample_map.csv').rename(columns={'Sample': 'serum'})

In [ ]:
for smpl in smpls.keys():
    display(HTML(smpls[smpl].head().to_html(index=False)))

#### Add plate number to fraction infectivity df

In [ ]:
dfs={}
for rnd in fidfs.keys():
    dfs[rnd] = fidfs[rnd].merge(smpls[rnd], how='outer', on='serum').drop(['Virus', 'PlateLayout', 'StartDil', 'DilFactor'], axis=1)

In [ ]:
for rnd in dfs.keys():
    display(HTML(dfs[rnd].head().to_html(index=False))

### Fix Plate 4 Issue for Rnd1

I forgot to discard the last 30 uL from the serum dilutions in row H of plate 4 in Rnd1, so I need to drop the final dilution for all plate 

In [ ]:
dfs['rnd1'].drop(dfs['rnd1'][(dfs['rnd1']['Plate']=='Plate4')&(dfs['rnd1']['concentration']==(0.05/(3**6)))].index, axis=0, inplace=True)
dfs['rnd1'] = dfs['rnd1'].sort_values(['Plate', 'SampleNum', 'replicate'])
display(HTML(dfs['rnd1'].to_html(index=False)))

## Calculate Fits using Neut Curve

For some reason must drop serum pool from rnd2.

In [ ]:
display(HTML(dfs['rnd2'][dfs['rnd2']['serum'] == '2017-2018 serum pool'].to_html(index=False)))

In [ ]:
dfs['rnd2'] = dfs['rnd2'][dfs['rnd2']['serum'] != '2017-2018 serum pool']

In [ ]:
fits = {}
fit_params = {}

for rnd in dfs.keys():
    fits[rnd] = neutcurve.CurveFits(dfs[rnd], fixtop=True)
    fit_params[rnd] = fits[rnd].fitParams(ics=[50, 90])
    fit_params[rnd]['nt50'] = 1/fit_params[rnd]['ic50']
    fit_params[rnd]['nt90'] = 1/fit_params[rnd]['ic90']

In [ ]:
for rnd in fit_params.keys():
    display(HTML(fit_params[rnd].head().to_html(index=False)))


### Merge fit parameter data and sample data

In [ ]:
fits_dfs = {}

for rnd in fit_params.keys():
    fits_dfs[rnd] = pd.merge(fit_params[rnd], smpls[rnd], how='outer', on='serum')
    fits_dfs[rnd].drop(['PlateLayout', 'Virus', 'DilFactor', 'StartDil'], axis=1, inplace=True)

In [ ]:
for rnd in fits_dfs.keys():
    display(HTML(fits_dfs[rnd].head().to_html(index=False))

### Find any samples with interpolated IC50s

Note that these samples were re-run in later rounds of neuts

In [ ]:
for rnd in fits_dfs.keys():
    print(rnd)
    display(HTML(fits_dfs[rnd][fits_dfs[rnd]['ic50_bound']=='upper'].to_html(index=False)))

## Plot all neut curves

In [ ]:
for rnd in fits.keys():
    fig, axes = fits[rnd].plotSera(xlabel='concentration/dilution')

## Plot Ty1-Fc control neut curves

In [ ]:
curve = fits['rnd1'].getCurve(serum='Ty1-Fc', virus='S-d21-D614G', replicate='1')
print(f"The IC50 is {curve.ic50():.3g}")
fig, ax = curve.plot()

In [ ]:
curve = fits['rnd2'].getCurve(serum='Ty1-FC', virus='S-d21-D614G', replicate='1')
print(f"The IC50 is {curve.ic50():.3g}")
fig, ax = curve.plot()

In [ ]:
curve = fits['rnd3'].getCurve(serum='Ty1', virus='S-d21-D614G', replicate='average')
print(f"The IC50 is {curve.ic50():.3g}")
fig, ax = curve.plot()

In [ ]:
curve = fits['rnd4'].getCurve(serum='Ty1', virus='S-d21-D614G', replicate='average')
print(f"The IC50 is {curve.ic50():.3g}")
fig, ax = curve.plot()

In [ ]:
curve = fits['rnd5'].getCurve(serum='Ty1', virus='S-d21-D614G', replicate='average')
print(f"The IC50 is {curve.ic50():.3g}")
fig, ax = curve.plot()

In [ ]:
curve = fits['rnd6'].getCurve(serum='Ty1-Fc', virus='S-d21-D614G', replicate='average')
print(f"The IC50 is {curve.ic50():.3g}")
fig, ax = curve.plot()

In [ ]:
curve = fits['rnd1'].getCurve(serum='20/130 Ref', virus='S-d21-D614G', replicate='average')
print(f"The IC50 is {1/curve.ic50():.3g}")
fig, ax = curve.plot()

In [ ]:
curve = fits['rnd3'].getCurve(serum='Ref 20/130', virus='S-d21-D614G', replicate='average')
print(f"The IC50 is {1/curve.ic50():.3g}")
fig, ax = curve.plot()

In [ ]:
curve = fits['rnd5'].getCurve(serum='Ref 20/130', virus='S-d21-D614G', replicate='average')
print(f"The IC50 is {1/curve.ic50():.3g}")
fig, ax = curve.plot()

## Combine data into one df


In [ ]:
all_data_df = pd.concat(fits_dfs)

## Drop Earlier Data for Samples that were re-run

Drop first sample and only keep re-run for samples that were re-run.

In [ ]:
rerun = ['0829-1', '0879-1', '0858-1', '0854-1', '0856-1', '0805-1', '0818-1',
         '0817-1', '0835-1', '0873-1', '0866-1', '0824-1', '0834-1']

In [ ]:
print(len(all_data_df))
drop_idxs = []
for sample in rerun:
    sample_data = all_data_df[all_data_df['serum']==sample]
    drop_idxs.append(sample_data['DateSetUp'].idxmin()) # drop earlier run

cleaned_data_df = all_data_df.drop(drop_idxs).reset_index(drop=True)
display(HTML(cleaned_data_df.head().to_html(index=False)))
print(len(cleaned_data_df))

### Output relevant data to csv

Output columns: `serum`, `ic50`, `ic90`, `nt50`, `nt90`, `DateSetUp`, `Plate`, `SampleNum`

In [ ]:
export_df_all = all_data_df[['serum', 'ic50', 'ic90', 'nt50', 'nt90', 'DateSetUp', 'Plate', 'SampleNum']]
export_df_clean = cleaned_data_df[['serum', 'ic50', 'ic90', 'nt50', 'nt90', 'DateSetUp', 'Plate', 'SampleNum']]

# `all_neut_results` includes all neuts, including the first neuts
# from samples I reran and all controls.
export_df_all.to_csv('./all_neut_results.csv') 
# `mouse_plus_ctrls_neuts` only includes the later neuts for the
# samples I re-ran and all controls.
export_df_clean.to_csv('./mouse_plus_ctrls_neuts.csv')

## Initial Analyses

Only including re-runs for samples I ran twice.
Not including controls.

Export csv after adding timepoint and mouse data.

In [ ]:
drop_samples = ['GF-8', 'Ref 20/130', '2017-2018 Serum Pool', '2017-2018 Pool', '20/130 Ref', '2017-2018 serum pool', 'Ty1', 'Ty1-FC', 'Ty1-Fc']
cleaned_data_df = cleaned_data_df[~cleaned_data_df['serum'].isin(drop_samples)].copy()
cleaned_data_df['Timepoint'] = cleaned_data_df['serum'].apply(lambda x: 'Prime' if '-1' in x else 'Boost')
cleaned_data_df['Mouse'] = cleaned_data_df['serum'].apply(lambda x: x[:-2])

In [ ]:
display(cleaned_data_df.sort_values('Mouse').reset_index(drop=True).head())

In [ ]:
groups = {'Group 1': ['0840', '0837', '0839', '0794', '0795', '0793'], 
          'Group 2': ['0801', '0803', '0805', '0850', '0848', '0846'],
          'Group 3': ['0808', '0807', '0810', '0853', '0854', '0852'],
          'Group 4': ['0813', '0812', '0815', '0856', '0858', '0859'],
          'Group 5': ['0816', '0818', '0817', '0864', '0861', '0863'],
          'Group 6': ['0835', '0831', '0834', '0867', '0866', '0870'],
          'Group 7': ['0827', '0828', '0829', '0874', '0873', '0872'],
          'Group 8': ['0823', '0824', '0825', '0879', '0877', '0876']}

In [ ]:
cleaned_data_df["Group"] = cleaned_data_df["Mouse"].apply(lambda x: [group for group in groups.keys() if x in groups[group]][0])

In [ ]:
display(cleaned_data_df.head())

### Export csv of cleaned mouse neut data

Only includes latest run for samples I re-ran and doesn't include human serum (e.g. naive serum, ref serum or GF-8 sample) or Ty1-FC controls.

In [ ]:
export_df_clean_noctrls = cleaned_data_df[['serum', 'ic50', 'ic90', 'nt50', 'nt90', 'DateSetUp', 'Plate', 'SampleNum', 'Timepoint', 'Mouse', 'Group']]
# `mouse_neuts` only includes the mouse neuts (no controls) and 
# only includes the later data for the samples I re-ran
export_df_clean_noctrls.to_csv('./mouse_neuts.csv')

### Initial plotting

In [ ]:
group_list = ['Group 1', 'Group 2', 'Group 3', 'Group 4', 'Group 5',
              'Group 6', 'Group 7', 'Group 8']
group_cat = pd.Categorical(cleaned_data_df['Group'], categories=group_list)

tp_list = ['Prime', 'Boost']
tp_cat = pd.Categorical(cleaned_data_df['Timepoint'], categories=tp_list)

# assign to a new column in the DataFrame
cleaned_data_df = cleaned_data_df.assign(group_order = group_cat)
cleaned_data_df = cleaned_data_df.assign(timepoint_order = tp_cat)

In [ ]:
ic50s_plot = (ggplot(cleaned_data_df, aes('group_order', 'nt50', color='tp_cat')) +
              geom_boxplot(outlier_alpha=0) +
              geom_point(size=2, alpha=0.5, position=position_dodge(width=0.75)) +
              scale_color_manual(values=CBP) +
              theme(axis_text_x=element_text(angle=90, vjust=1, hjust=0.5),
                    figure_size=(8, 6)) +
#               facet_wrap('~ timepoint_order') +
              geom_hline(yintercept=20, color='grey', linetype='dashed') +
              scale_y_continuous(trans='log10') +
              labs(color='Timepoint')
             )

_ = ic50s_plot.draw()

### Convert to Markdown

In [ ]:
!jupyter nbconvert MouseNeuts.ipynb --to markdown